# Feature Engineering for Per-Query Optimization

Notes:

* searcharray produced an error with numpy version 2.1.2
* Make sure to use numpy==1.26.4

Error reported to be fixed, not verified

In [1]:
import pandas as pd
import math
import string
import requests
import json
from searcharray import SearchArray
import numpy as np

In [2]:
df_metrics = pd.read_csv('metrics.csv')

In [3]:
df_metrics[df_metrics['metric'] == 'ndcg'].head(3)

,query,pipeline,metric,value
14520,$30 roblox gift card not digital,l2arithmetic_mean0.0,ndcg,0.075767
14521,$30 roblox gift card not digital,l2arithmetic_mean0.1,ndcg,0.075767
14522,$30 roblox gift card not digital,l2arithmetic_mean0.2,ndcg,0.075767


In [4]:
indexes = df_metrics[df_metrics['metric'] == 'ndcg'].groupby(by='query')['value'].idxmax()
df_filtered = df_metrics.loc[indexes, ['query','pipeline', 'value']]
df_filtered.reset_index(inplace=True, drop=True)

In [48]:
df_filtered.head(3)

,query_string,ndcg,neuralness,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9
0,$30 roblox gift card not digital,0.075767,1.0,6,32,1,1,0,0.000000,0.000000,0.723621,0.690628
1,(fiction without frontiers),0.451875,0.9,3,27,0,1,42,0.163126,1.418506,0.730339,0.693966
2,100,0.332592,0.8,1,3,1,0,220506,0.187512,1.249529,1.000000,0.637586


In [6]:
def get_neuralness(pipeline):
    keywordness = pipeline[-3:]

    try:
        neuralness = 1 - float(keywordness)
        return neuralness
    except ValueError:
        print("ERROR")
        return math.nan

In [7]:
df_filtered['neuralness'] = df_filtered.pipeline.apply(get_neuralness)

In [8]:
df_filtered = df_filtered.drop(columns=['pipeline'])

In [9]:
df_filtered.columns = ['query_string', 'ndcg', 'neuralness']

In [49]:
df_filtered.head(3)

,query_string,ndcg,neuralness,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9
0,$30 roblox gift card not digital,0.075767,1.0,6,32,1,1,0,0.000000,0.000000,0.723621,0.690628
1,(fiction without frontiers),0.451875,0.9,3,27,0,1,42,0.163126,1.418506,0.730339,0.693966
2,100,0.332592,0.8,1,3,1,0,220506,0.187512,1.249529,1.000000,0.637586


## Query Feature Functions

`num_of_terms`: we expect the number of query terms to be a good feature because the more query terms used the more specific a query is.

`query_length`: we expect the query length (number of characters) to be a good feature because the longer a query is the more specific it is.

`has_numbers`: we expect the presence of numbers/digits to be a good feature because numbers may point towards specific queries.

`has_special_char`: we expect the presence of special characters (non-alphanumeric characters) to be a good feature because that may point towards specific queries. 

In [11]:
def num_of_terms(query_string):
    terms = query_string.split(" ")
    return len(terms)

def query_length(query_string):
    return len(query_string)

def has_numbers(query_string):
    return int(any(char.isdigit() for char in query_string))

def has_special_char(query_string):
    # Define special characters (all non-alphanumeric characters)
    special_chars = string.punctuation
    # Return True if any character in the string is a special character
    return int(any(char in special_chars for char in query_string))

In [12]:
df_filtered['num_of_terms'] = df_filtered.query_string.apply(num_of_terms)
df_filtered['query_length'] = df_filtered.query_string.apply(query_length)
df_filtered['has_numbers'] = df_filtered.query_string.apply(has_numbers)
df_filtered['has_special_char'] = df_filtered.query_string.apply(has_special_char)

In [13]:
df_filtered.head(5)

,query_string,ndcg,neuralness,num_of_terms,query_length,has_numbers,has_special_char
0,$30 roblox gift card not digital,0.075767,1.0,6,32,1,1
1,(fiction without frontiers),0.451875,0.9,3,27,0,1
2,100,0.332592,0.8,1,3,1,0
3,10x10x6 cake box without window,0.099900,1.0,5,31,1,0
4,15 inch light weight laptop that has lots of m...,0.039399,0.0,28,125,1,1


## Result Feature Functions

`num_results_kw_search`: the number of results for the keyword query.

`max_doc_score`: max score of the retrieved top 10 titles. The scores are BM25 scores calculated individually per result set. That means that the document frequencies of terms is not calculated on the whole index but only on the retrieved subset for the query, making the scores more comparable to each other.

`sum_of_doc_scores`: we use the sum of the scores (and no average value) as an aggregate to have a measure of how relevant all retrieved top 10 titles are.

`get_semantic_scores_for_query`: retrieves the semantic scores for the neural query for the top 10. We store the maximum and the mean as result features.

### Helper functions

`get_titles_for_query`: retrieves the titles of the keyword matches for later score calculation

`whitespace_to_lower`: tokenizes and lowercases text

`get_model_id`: to run a neural search to get the semantic scores we need to know the id of the available model in OpenSearch


### Get Number of Results for Keyword Search

In [14]:
def num_results_kw_search(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
    
    
    
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ]
      },
      "query": {
        "multi_match" : {
          "type":       "best_fields",
          "fields":     [
            "product_id^100",
            "product_bullet_point^3",
            "product_color^2",
            "product_brand^5",
            "product_description",
            "product_title^10"
          ],
          "operator":   "and",
          "query":      query_string
        }
      },
      "track_total_hits": "true"
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    num_results = response['hits']['total']['value']
    return num_results

In [15]:
def get_titles_for_query(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
      
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ],
        "includes": "product_title"
      },
      "query": {
        "multi_match" : {
          "type":       "best_fields",
          "fields":     [
            "product_id^100",
            "product_bullet_point^3",
            "product_color^2",
            "product_brand^5",
            "product_description",
            "product_title^10"
          ],
          "operator":   "and",
          "query":      query_string
        }
      },
      "track_total_hits": "true"
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    titles = []
    num_results = response['hits']['total']['value']
    if num_results > 0:
        # get all titles
        titles = [item['_source']['product_title'] for item in response['hits']['hits']]

    df = pd.DataFrame(titles, columns=['title'])

    return df

In [16]:
df_filtered['num_results'] = df_filtered.query_string.apply(num_results_kw_search)

### Tokenizer Definition

We define a simple tokenizer that splits on whitespaces and lowercases

In [17]:
def whitespace_to_lower(text):
    split = text.lower().split()
    return [token
        for token in split]

ToDo: make the following functions more efficient.

The following functions contain redundant code and only differ in the return value.

In [18]:
# we use searcharray to calculate the highest title score among the retrieved docs
def max_doc_score(df, query):
    # if we retrieved zero docs we return 0 as the max score
    if df.shape[0] == 0:
        return 0
    else:
        # index the titles with the defined tokenizer
        df['title_indexed'] = SearchArray.index(df['title'], tokenizer=whitespace_to_lower)
        # tokenize the query with the same tokenizer
        tokenized_query = df['title_indexed'].array.tokenizer(query)
        # calculate the score for each query term
        # the result is a matrix with number of titles x number of query term dimensions
        scores = np.asarray([df['title_indexed'].array.score(query_term)
                                    for query_term in tokenized_query])
        # calculate the sum of all query terms per title
        sums = [sum(group) for group in zip(*scores)]
        # return the maximum
        return max(sums)

# we use searcharray to calculate the sum of all title scores among the retrieved docs
def sum_of_doc_scores(df, query):
    # if we retrieved zero docs we return 0 as the max score
    if df.shape[0] == 0:
        return 0
    else:
        # index the titles with the defined tokenizer
        df['title_indexed'] = SearchArray.index(df['title'], tokenizer=whitespace_to_lower)
        # tokenize the query with the same tokenizer
        tokenized_query = df['title_indexed'].array.tokenizer(query)
        # calculate the score for each query term
        # the result is a matrix with number of titles x number of query term dimensions
        scores = np.asarray([df['title_indexed'].array.score(query_term)
                                    for query_term in tokenized_query])
        # calculate the sum of all query terms per title
        sums = [sum(group) for group in zip(*scores)]
        # return the sum of all title scores
        return sum(sums)

### Get Semantic Scores

First we get the model id, then we use the model id in the query to retrieve the semantic scores.

In [19]:
# Get model_id
# We are assuming that the installation has only one model. Change this if you have more models 
# and need to pick a specific one

def get_model_id():
    url = "http://localhost:9200/_plugins/_ml/models/_search"
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    payload = {
      "query": {
        "match_all": {}
      },
      "size": 1
    }
    
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

    return response.json()['hits']['hits'][0]['_source']['model_id']

model_id = get_model_id()

In [20]:
def get_semantic_scores_for_query(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
    
    
    
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ],
        "includes": "product_title"
      },
      "query": {
        "neural": {
          "title_embedding": {
            "query_text": query_string,
            "k": 100,
            "model_id": model_id
          }
        }
      },
      "size": 10
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    titles = []
    num_results = response['hits']['total']['value']
    if num_results > 0:
        # get all titles
        scores = [item['_score'] for item in response['hits']['hits']]

    return scores

In [21]:
df_filtered['max_title_score'] = df_filtered['query_string'].apply(lambda q: max_doc_score(get_titles_for_query(q), q))
df_filtered['sum_title_scores'] = df_filtered['query_string'].apply(lambda q: sum_of_doc_scores(get_titles_for_query(q), q))

2024-10-28 17:23:18,582 - searcharray.indexing - INFO - Indexing begins w/ 4 workers
2024-10-28 17:23:18,583 - searcharray.indexing - INFO - 0 Batch Start tokenization
2024-10-28 17:23:18,588 - searcharray.indexing - INFO - Tokenizing 10 documents
2024-10-28 17:23:18,590 - searcharray.indexing - INFO - Tokenization -- vstacking
2024-10-28 17:23:18,591 - searcharray.indexing - INFO - Tokenization -- DONE
2024-10-28 17:23:18,592 - searcharray.indexing - INFO - Inverting docs->terms
2024-10-28 17:23:18,593 - searcharray.indexing - INFO - Encoding positions to bit array
2024-10-28 17:23:18,594 - searcharray.indexing - INFO - Batch tokenization complete
2024-10-28 17:23:18,595 - searcharray.indexing - INFO - (main thread) Processing 1 batch results
2024-10-28 17:23:18,596 - searcharray.indexing - INFO - Indexing from tokenization complete
2024-10-28 17:23:18,616 - searcharray.indexing - INFO - Indexing begins w/ 4 workers
2024-10-28 17:23:18,617 - searcharray.indexing - INFO - 0 Batch Start

In [22]:
df_filtered['max_semantic_score'] = df_filtered['query_string'].apply(lambda q: max(get_semantic_scores_for_query(q)))
df_filtered['avg_semantic_score'] = df_filtered['query_string'].apply(lambda q: np.mean(get_semantic_scores_for_query(q)))

In [23]:
df_filtered.head(5)

,query_string,ndcg,neuralness,num_of_terms,query_length,has_numbers,has_special_char,num_results,max_title_score,sum_title_scores,max_semantic_score,avg_semantic_score
0,$30 roblox gift card not digital,0.075767,1.0,6,32,1,1,0,0.000000,0.000000,0.723621,0.690628
1,(fiction without frontiers),0.451875,0.9,3,27,0,1,42,0.163126,1.418506,0.730339,0.693966
2,100,0.332592,0.8,1,3,1,0,220506,0.187512,1.249529,1.000000,0.637586
3,10x10x6 cake box without window,0.099900,1.0,5,31,1,0,0,0.000000,0.000000,0.637134,0.614345
4,15 inch light weight laptop that has lots of m...,0.039399,0.0,28,125,1,1,0,0.000000,0.000000,0.656478,0.611948


In [24]:
df_filtered.columns

Index(['query_string', 'ndcg', 'neuralness', 'num_of_terms', 'query_length',
       'has_numbers', 'has_special_char', 'num_results', 'max_title_score',
       'sum_title_scores', 'max_semantic_score', 'avg_semantic_score'],
      dtype='object')

In [25]:
# rename the columns
df_filtered = df_filtered.rename(columns={"num_of_terms": "f_1",
                            "query_length": "f_2",
                            "has_numbers": "f_3",
                            "has_special_char": "f_4",
                            "num_results": "f_5",
                            "max_title_score": "f_6",
                            "sum_title_scores": "f_7",
                            "max_semantic_score": "f_8",
                            "avg_semantic_score": "f_9"
                           })

## Explore Linear Regression Models with Different Feature Combinations

In [35]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [f'f_{i}' for i in range(1, 10)]
y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
model = RandomForestRegressor(random_state=42)

# Custom RMSE scorer
#def get_root_mean_squared_error(y_true, y_pred):
#    return root_mean_squared_error(y_true, y_pred, squared=False)

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('-inf')
best_combination = None
best_metrics = {}

# Try all feature combinations of length 2 to 9
for r in range(2, len(feature_columns) + 1):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset
        
        # Perform cross-validation
        scores = cross_val_score(model, X, y, cv=cv)
        rmse_scores = cross_val_score(model, X, y, cv=cv, scoring=rmse_scorer)
        
        # Calculate the mean metrics for the current combination
        mean_score = scores.mean()
        std_score = scores.std()
        mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization
        print(f"feature combo: {combo}, score: {mean_rmse}")
        
        
        # Update if the current combination is better
        if mean_score > best_score:
            best_score = mean_score
            best_combination = combo
            best_metrics = {
                'Accuracy (mean)': mean_score,
                'Accuracy (std dev)': std_score,
                'RMSE (mean)': mean_rmse
            }

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Metrics:")
print(f"Accuracy (mean): {best_metrics['Accuracy (mean)']}")
print(f"Accuracy (std dev): {best_metrics['Accuracy (std dev)']}")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

feature combo: ('f_1', 'f_2'), score: 0.3800054728713641
feature combo: ('f_1', 'f_3'), score: 0.32927153161063794
feature combo: ('f_1', 'f_4'), score: 0.32587391296691226
feature combo: ('f_1', 'f_5'), score: 0.36654632379129076
feature combo: ('f_1', 'f_6'), score: 0.37138491810214536
feature combo: ('f_1', 'f_7'), score: 0.3855394507766348
feature combo: ('f_1', 'f_8'), score: 0.35548331712732645
feature combo: ('f_1', 'f_9'), score: 0.3675731281513138
feature combo: ('f_2', 'f_3'), score: 0.36304035340964536
feature combo: ('f_2', 'f_4'), score: 0.3483090012128046
feature combo: ('f_2', 'f_5'), score: 0.36342619249673086
feature combo: ('f_2', 'f_6'), score: 0.3845851923153947
feature combo: ('f_2', 'f_7'), score: 0.38259074982893526
feature combo: ('f_2', 'f_8'), score: 0.36178244095605877
feature combo: ('f_2', 'f_9'), score: 0.36817549542795275
feature combo: ('f_3', 'f_4'), score: 0.3317760701463717
feature combo: ('f_3', 'f_5'), score: 0.4026574910282884
feature combo: ('f_3'

## Train a Random Forest Regression Model with cross-validation

In [38]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
import numpy as np

# Prepare features and target
feature_columns = [f'f_{i}' for i in range(1, 10)]
X = df_filtered[feature_columns]   # Select feature columns
y = df_filtered['neuralness']      # Select the target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
model = RandomForestRegressor(random_state=42)

# Perform cross-validation
scores = cross_val_score(model, X, y, cv=cv)
rmse_scorer = make_scorer(root_mean_squared_error)

rmse_scores = cross_val_score(model, X, y, cv=cv, scoring=rmse_scorer)

# Output the results
print("Accuray:", scores.mean())
print("Standard deviation:", scores.std())
print("Cross-Validation Root Mean Squared Error:", np.mean(rmse_scores))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R^2 Score:", r2)

Accuray: -0.12377425503111397
Standard deviation: 0.08182795705905409
Cross-Validation Root Mean Squared Error: 0.3392405950378962
Root Mean Squared Error: 0.31486909256792006
R^2 Score: -0.40522708836664445


## Explore Linear Regression Models with Different Feature Combinations

In [33]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [f'f_{i}' for i in range(1, 10)]
y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
regr = LinearRegression()

# Custom RMSE scorer
#def get_root_mean_squared_error(y_true, y_pred):
#    return root_mean_squared_error(y_true, y_pred, squared=False)

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('-inf')
best_combination = None
best_metrics = {}

# Try all feature combinations of length 2 to 9
for r in range(2, len(feature_columns) + 1):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset
        
        # Perform cross-validation
        scores = cross_val_score(regr, X, y, cv=cv)
        rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)
        
        # Calculate the mean metrics for the current combination
        mean_score = scores.mean()
        std_score = scores.std()
        mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization
        print(f"feature combo: {combo}, score: {mean_rmse}")
        
        
        # Update if the current combination is better
        if mean_score > best_score:
            best_score = mean_score
            best_combination = combo
            best_metrics = {
                'Accuracy (mean)': mean_score,
                'Accuracy (std dev)': std_score,
                'RMSE (mean)': mean_rmse
            }

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Metrics:")
print(f"Accuracy (mean): {best_metrics['Accuracy (mean)']}")
print(f"Accuracy (std dev): {best_metrics['Accuracy (std dev)']}")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

feature combo: ('f_1', 'f_2'), score: 0.33081266885987115
feature combo: ('f_1', 'f_3'), score: 0.3298325966675315
feature combo: ('f_1', 'f_4'), score: 0.3303192482828764
feature combo: ('f_1', 'f_5'), score: 0.43397714135184307
feature combo: ('f_1', 'f_6'), score: 0.33237093516763605
feature combo: ('f_1', 'f_7'), score: 0.3319367773508479
feature combo: ('f_1', 'f_8'), score: 0.3264977451979067
feature combo: ('f_1', 'f_9'), score: 0.33085540158718924
feature combo: ('f_2', 'f_3'), score: 0.3258265671940376
feature combo: ('f_2', 'f_4'), score: 0.3265380785153506
feature combo: ('f_2', 'f_5'), score: 0.4325717723162904
feature combo: ('f_2', 'f_6'), score: 0.328666093338427
feature combo: ('f_2', 'f_7'), score: 0.3278073172028537
feature combo: ('f_2', 'f_8'), score: 0.3231254008623167
feature combo: ('f_2', 'f_9'), score: 0.32671396206917913
feature combo: ('f_3', 'f_4'), score: 0.32230826705117666
feature combo: ('f_3', 'f_5'), score: 0.42690636281149363
feature combo: ('f_3', 'f

## Train a Linear Regression Model with cross-validation

In [39]:
from sklearn.linear_model import LinearRegression

# Prepare features and target
feature_columns = [f'f_{i}' for i in range(1, 10)]
X = df_filtered[feature_columns]   # Select feature columns
y = df_filtered['neuralness']      # Select the target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
regr = LinearRegression()

# Perform cross-validation
scores = cross_val_score(regr, X, y, cv=cv)
rmse_scorer = make_scorer(root_mean_squared_error)

rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)

# Output the results
print("Accuray:", scores.mean())
print("Standard deviation:", scores.std())
print("Cross-Validation Root Mean Squared Error:", np.mean(rmse_scores))

regr.fit(X_train, y_train)

y_regr_pred = regr.predict(X_test)

# Evaluate the model
rmse = root_mean_squared_error(y_test, y_regr_pred)
r2 = r2_score(y_test, y_regr_pred)

print("Root Mean Squared Error:", rmse)
print("R^2 Score:", r2)

Accuray: -3.104349443117165
Standard deviation: 5.985149030520057
Cross-Validation Root Mean Squared Error: 0.5029819666786967
Root Mean Squared Error: 0.265534111435833
R^2 Score: 0.0006282059116358152


In [40]:
vector = [[3, 21, 0, 0, 22, 1.239339, 3.411745, 0.600457, 0.550430]]
df_test = pd.DataFrame(vector, columns=["f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9"])
df_test

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9
0,3,21,0,0,22,1.239339,3.411745,0.600457,0.55043


In [41]:
model.predict(df_test)

array([0.457])

In [42]:
regr.predict(df_test)

array([0.72486381])

In [43]:
X_test.shape[0]

44

In [44]:
X_test.index

Index([132, 148,  93, 180,  15, 115, 172, 209,  75, 142, 100,  30, 190,   9,
        67, 218, 175,  18, 197,  66, 189,  96, 210,  25, 127, 201,  97,  16,
        45, 154, 111,  55, 108, 196,  73, 109,  82, 138, 141, 101,  86, 193,
       120,  95],
      dtype='int64')

In [45]:
# Store the test data
df_filtered.loc[df_filtered.index.isin(X_test.index)].to_csv('dynamic_optimizer_test_data.csv', index=False)

## Save Models

In [46]:
import pickle

with open('regression_model.pkl', 'wb') as fid:
    pickle.dump(regr, fid) 

with open('random_forest_model.pkl', 'wb') as fid:
    pickle.dump(model, fid) 